In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

In [2]:
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

In [3]:
def read_data(filename):
    theta = pd.read_csv(filename, header = None)
    theta = theta[[0, 1, 2, 3, 9, 10]]
    theta[0] = theta[0].apply(lambda x: int(x.replace("(", "")))
    theta[10] = theta[10].apply(lambda x: float(x.replace(")", "")))
    for i in [2,3]:
        theta[i] = theta[i].apply(lambda x: int(x))
    theta[9] = theta[9].apply(lambda x: float(x))
    theta = theta.sort_values(by=[0, 2, 3])
    theta = theta.rename(columns={0:"partitions", 1:"condition", 2:"attrIndex1", 3:"attrIndex2", 9:"mean", 10:"std"})
    return theta

In [4]:
def extract_joins(theta_dataset, attrIndex1, attrIndex2):
    theta_extr = theta_dataset[(theta_dataset["attrIndex1"]==attrIndex1) & (theta_dataset["attrIndex2"]==attrIndex2)]
    theta_extr_less = theta_extr[theta_extr["condition"]=="<"].sort_values(by="partitions")
    theta_extr_more = theta_extr[theta_extr["condition"]==">"].sort_values(by="partitions")
    return theta_extr_less, theta_extr_more

In [5]:
def plot_dataset(theta_less, theta_more, size_label):
    plt.figure(figsize=(9, 5))

    # Plot data and save figure
    markers, caps, bars = plt.errorbar(theta_less["partitions"], theta_less["mean"], yerr=theta_less["std"], label = "$<$ condition",
                                       color="blue", marker="D", markersize=7)
    [bar.set_alpha(0.5) for bar in bars]
    markers, caps, bars = plt.errorbar(theta_more["partitions"], theta_more["mean"], yerr=theta_more["std"], label = "$>$ condition",
                                       color="orange", marker="o", markersize=7)
    [bar.set_alpha(0.5) for bar in bars]
    plt.grid(linestyle='dotted')

    # set labels (LaTeX can be used) -> Note: with the setting deactivated, this will print \textbf{...}
    plt.xlabel(r'\textbf{Partitions number}', fontsize=11)
    plt.ylabel(r'\textbf{Query execution time [s] }', fontsize=11)
    plt.title(r'\textbf{Execution time change with number of partitions - ' + size_label + ' datasets}', fontsize=13.5)
    plt.legend()
    plt.xscale("log")

    plt.savefig("plot/time_change" + size_label + ".pdf")
    plt.close()

In [33]:
def get_best(theta_less, theta_more):
    best_less = theta_less.sort_values(by="mean").iloc[0]
    best_more = theta_more.sort_values(by="mean").iloc[0]
    return best_less, best_more

In [7]:
theta_4000 = read_data("../thetajoin/thetajoin_count_4000.txt")

In [8]:
theta_4000_2_2_less, theta_4000_2_2_more = extract_joins(theta_4000, 2, 2)

In [58]:
plot_dataset(theta_4000_2_2_less, theta_4000_2_2_more, "4K")

In [59]:
labels = ["1K", "2K", "3K", "4K"]
for index, i in enumerate(range(1000, 4001, 1000)):
    filename = "../thetajoin/thetajoin_count_" + str(i) + ".txt"
    theta = read_data(filename)
    theta_less, theta_more = extract_joins(theta, 2, 2)
    plot_dataset(theta_less, theta_more, labels[index])

In [37]:
for index, i in enumerate(range(1000, 4001, 1000)):
    filename = "../thetajoin/thetajoin_count_" + str(i) + ".txt"
    theta = read_data(filename)
    theta_less, theta_more = extract_joins(theta, 2, 2)
    best_less, best_more = get_best(theta_less, theta_more)
    print(i)
    print("Best less:")
    print(best_less)
    print("Best more:")
    print(best_more)

1000
Best less:
partitions           16
condition             <
attrIndex1            2
attrIndex2            2
mean           0.224862
std           0.0197235
Name: 51, dtype: object
Best more:
partitions           32
condition             >
attrIndex1            2
attrIndex2            2
mean           0.222244
std           0.0126878
Name: 62, dtype: object
2000
Best less:
partitions           16
condition             <
attrIndex1            2
attrIndex2            2
mean           0.268458
std           0.0257556
Name: 60, dtype: object
Best more:
partitions          16
condition            >
attrIndex1           2
attrIndex2           2
mean          0.256345
std           0.013926
Name: 10, dtype: object
3000
Best less:
partitions           64
condition             <
attrIndex1            2
attrIndex2            2
mean           0.324203
std           0.0179939
Name: 43, dtype: object
Best more:
partitions           32
condition             >
attrIndex1            2
attrIndex2   